In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_data = pd.read_csv("./data/train_data.csv")
sample_submission = pd.read_csv("./data/sample_submission.csv")
turb_location = pd.read_csv("./data/turb_location.CSV")

# ARMA